# get Cause - efffect phrase from the sentence.

In [1]:
#filtered_rows.reset_index(drop=True, inplace=True)


## 1. load cause-effect data

In [2]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

#'data/in/semeval/sem_CE_test.csv'
#'data/in/RAG/ADE/ADE_CE_extract_TEST.csv')
#'data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUT.csv'


pm_DB = pd.read_csv('data/in/KNN_RAG/SCITE_CE_extract_INPUT.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]
instances = pm_DB['sentence'].to_list()
len(instances)

191

## 2. load few shot database

In [3]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()
len(examples)

2365

In [4]:
semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

## 3. identify cause- effect phrase

In [5]:
#load causality input sentence.
#input_sentence_list = filtered_rows['sentence'].to_list()
#gpt to extract cause effect and a causality phrase.
#: few shot: example1: , causality phrase: .

In [6]:
import groq
from groq import Groq
client = groq.Groq(api_key= "enter your API key here..")#sameer key

#gemma2-9b-it# llama3-8b-8192
#mixtral-8x7b-32768
def C_E_CP_Identifier(causality_sentence, fewshot_examples):

    chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify all the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause1": "", "effect1": "", "causality_phrase1": "", "cause2": "", "effect2": "", "causality_phrase2": "",
          "cause3": "", "effect3": "", "causality_phrase3": "", ....}. Make sure your JSON ouput is closed with } at the end.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
          
          Rule 3: There could be multiple cause-effect phrases in the sentence. You should extract all 
          of them. For instance, consider the input sentence "Mutations in MCFD2 causing F5F8D are null
          mutations but also missense mutations, which cause destabilization of its flexible structure
          and the loss of its interaction with ERGIC-53 .". 
          Your identified output will be:
          {"cause1": "Mutations in MCFD2", "effect1": "F5F8D", "causality_phrase1": "causing", 
          "cause2": "missense mutations", "effect2": "destabilization of its flexible structure", "causality_phrase2": "cause",
          "cause3": "missense mutations", "effect3": "the loss of its interaction with ERGIC-53", "causality_phrase3": "cause"

          
          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
          

                   
          ''' + fewshot_examples
            
        },
        {
          "role": "user",
          "content": input_sentence 
        },
    ],

    # The language model which will generate the completion.
    model="mixtral-8x7b-32768",
    temperature=0.0,
    seed = 1,
    max_tokens=100,
    top_p=1,
    response_format={"type": "json_object"},)
    out = chat_completion.choices[0].message.content
    
    return out

In [7]:
def get_fewshotexamples_C_E_CP(fewshot_index):

    
    #fewshot_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120]
    
    
    train_sentence = semeval_data['sentence'].to_list()
    train_causalityPhrase = semeval_data['causality_phrase'].to_list()

    fewshot_example_sentence = [train_sentence[i] for i in fewshot_index]
    fewshot_example_causalityPhrase = [train_causalityPhrase[i] for i in fewshot_index]


    import json
    #fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_example_causalityPhrase[i]} 
                    for i, sentence in enumerate(fewshot_example_sentence)}
    
    # Assuming json_data is already defined
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_example_sentence, fewshot_example_causalityPhrase

In [8]:
# semeval
filtered_rows = pm_DB

In [9]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        #print('ind :', ind)
        window_size = len(p)

        # Slide through input_sentence with a window size 
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('causality phrase: ',p, 'index is', ind)
                #print(f"Window '{window}' - LS: {score}")

    #print('fewshot_index', fewshot_index)
    fewshot_index = list(set(fewshot_index))
    #print('fewshot_index', fewshot_index)
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [10]:
import json
import re

def close_json_string(json_string):
    
    
    # Remove entries where "cause", "effect", and "causality_phrase" are all empty
    pattern = re.compile(r'"cause\d+":\s*"",\s*"effect\d+":\s*"",\s*"causality_phrase\d+":\s*""\s*,?\s*')
    json_string = re.sub(pattern, '', json_string)
    
    # Remove incomplete index entries
    incomplete_index_pattern = re.compile(r'"cause\d+":\s*""\s*,?\s*')
    json_string = re.sub(incomplete_index_pattern, '', json_string)
    
    # Remove trailing comma if exists
    json_string = re.sub(r',\s*}$', '}', json_string.strip())
    json_string = re.sub(r',\s*$', '', json_string.strip())
    
    
    
    # Find the last complete index
    last_index = re.findall(r'"cause\d+":\s*"[^"]*",\s*"effect\d+":\s*"[^"]*",\s*"causality_phrase\d+":\s*"[^"]*"', json_string)
    
    # If found, truncate the JSON string after the last complete index
    if last_index:
        last_index_end_pos = json_string.rfind(last_index[-1]) + len(last_index[-1])
        json_string = json_string[:last_index_end_pos]
    
    # Count the number of opening and closing braces
    open_braces = json_string.count('{')
    close_braces = json_string.count('}')
    
    # Add missing closing braces if needed
    if open_braces > close_braces:
        json_string += '}' * (open_braces - close_braces)
    
    return json_string

In [11]:
import json
import json
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
filtered_rows = pm_DB

sent_cause_effect_list = []
causality_phrase_list = []

fewshotfound_inputsentence_index=[]
for i in range(len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    
    #KNN+RAG few shot examples augmentation.
    fewshot_index = get_fewshot_examples_index(input_sentence)############FEWSHOT
    
    fewshot_sentence_rag, fewshot_patterns_rag = get_fewshotexamples_C_E_CP(fewshot_index) #############fewshot
    #fewshot_examples =  get_randomfewshotexamples()##################random shot/zero shot
        
    #print('input_sentence : ', input_sentence)
    ###
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_sentence_knn = [examples[j] for j in most_similar_indices]
    fewshot_patterns_knn = [causality_phrase[p] for p in most_similar_indices]
    
    fewshot_sentence_comb = fewshot_sentence_rag+fewshot_sentence_knn
    fewshot_patterns_comb = fewshot_patterns_rag+fewshot_patterns_knn

    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_patterns_comb[i]} 
                    for i, sentence in enumerate(fewshot_sentence_comb)}
    
    fewshot_examples = json.dumps(fewshot_examples)   

    #print('fewshot_comb : ', fewshot_examples)
    
    #####end
    
    
    try:
    
        # output generation
        out = C_E_CP_Identifier(input_sentence, fewshot_examples)
        #print('out1 : ', out)

        out = close_json_string(out)
        print('out : ', out)

    
    # Attempt to parse the JSON string into a dictionary
        data = json.loads(out)
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue  # Skip to the next iteration if there is a JSON error

        
    except TypeError as e:
        print('TypeError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
        
    except KeyError as e:
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
    
    except groq.APIConnectionError as e:
        print('APIConnectionError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
        
    except groq.RateLimitError as e:
        print('RateLimitError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])    
        continue
        
    except groq.APIStatusError as e:
        print('APIStatusError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
    # Parse JSON string into a dictionary
    data = json.loads(out)

    # Initialize lists
    sent_cause_effect = []
    causality_phrase_row = []

    # Iterate through the keys and collect values
    i_out = 1
    while f'cause{i_out}' in data and f'effect{i_out}' in data and f'causality_phrase{i_out}' in data:
        sent_cause_effect.append((input_sentence, data[f'cause{i_out}'], data[f'effect{i_out}']))
        causality_phrase_row.append(data[f'causality_phrase{i_out}'])
        i_out += 1

    if sent_cause_effect == []:
        print('empty')
        try:
            sent_cause_effect.append((input_sentence, data['cause'], data['effect']))
            causality_phrase_row.append(data['causality_phrase'])
        
        except KeyError as e:
            print('key...')
            sent_cause_effect_list.append([])
            causality_phrase_list.append([])
            continue
        
        
    # Print the results
    print("sent_cause_effect:", sent_cause_effect)
    #print("causality_phrase_row:", causality_phrase_row)
    sent_cause_effect_list.append(sent_cause_effect)
    causality_phrase_list.append(causality_phrase_row)
    print('i:', i)
    
    #if i==3:
        #break;
    

out :  {
  "cause1": "using solar electricity instead of conventional electricity sources",
  "effect1": "reduction of pollution",
  "causality_phrase1": "helps to reduce",
  "cause2": "pollution",
  "effect2": "smog, acid rain, respiratory illness and global warming",
  "causality_phrase2": "that causes"}
sent_cause_effect: [('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'using solar electricity instead of conventional electricity sources', 'reduction of pollution'), ('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'pollution', 'smog, acid rain, respiratory illness and global warming')]
i: 0
out :  {
  "cause": "capital murders",
  "effect": "execution",
  "causality_phrase": "that result in"
}
empty
sent_cause_effect: [('Thus, eva

out :  {
  "cause": "the eruption of Mt. Pinatubo",
  "effect": "an evacuation",
  "causality_phrase": "after"
}
empty
sent_cause_effect: [('After the war, as the Midway was preparing for retirement, she was called upon one last time to the Philippines to help with  an evacuation  after  the eruption  of Mt. Pinatubo.', 'the eruption of Mt. Pinatubo', 'an evacuation')]
i: 24
out :  {"cause": "the construction", "effect": "are intolerable", "causality_phrase": "produced by"}
empty
sent_cause_effect: [(' The dust ,  noise , and  sleep deprivation  produced by  the construction  are intolerable.', 'the construction', 'are intolerable')]
i: 25
out :  {
  "cause1": "the sun",
  "effect1": "skin irritation",
  "causality_phrase1": "from the",
  "cause2": "environmental pollutants",
  "effect2": "calms skin irritation",
  "causality_phrase2": "and calms from"}
sent_cause_effect: [('Oak root and chamomile provide a gentle astringent that helps reduce inflammation and calms  skin irritation  fr

out :  {"cause": "the purchase", "effect": "disappointment", "causality_phrase": "from the"}
empty
sent_cause_effect: [('In these cases,  the disappointment  from  the purchase  is not forgotten over time, but rather accumulates.', 'the purchase', 'disappointment')]
i: 47
out :  {
  "cause": "the drugs he sold",
  "effect": "the overdose death of Matthew Lessard, 19, of Lowell",
  "causality_phrase": "had caused"
}
empty
sent_cause_effect: [(' The drugs  he sold had caused  the overdose death  of Matthew Lessard, 19, of Lowell.', 'the drugs he sold', 'the overdose death of Matthew Lessard, 19, of Lowell')]
i: 48
out :  {
  "cause": "this staring effect",
  "effect": "Eye discomfort",
  "causality_phrase": "is exacerbated by"
}
empty
sent_cause_effect: [(' Eye discomfort  from  this staring effect  is exacerbated by low humidity.', 'this staring effect', 'Eye discomfort')]
i: 49
out :  {
  "cause": "loss of key employees",
  "effect": "big risk",
  "causality_phrase": "from"
}
empty
sen

out :  {
  "cause1": "the silversmiths",
  "effect1": "a riot breaks out",
  "causality_phrase1": "instigated by",
  "cause2": "Paul's evangelistic success",
  "effect2": "ruin their business",
  "causality_phrase2": "afraid that"}
sent_cause_effect: [("Before he leaves Ephesus, however,  a riot  breaks out, instigated by  the silversmiths  who manufacture idols of the goddess Artemis; they are afraid that Paul's evangelistic success will ruin their business.", 'the silversmiths', 'a riot breaks out'), ("Before he leaves Ephesus, however,  a riot  breaks out, instigated by  the silversmiths  who manufacture idols of the goddess Artemis; they are afraid that Paul's evangelistic success will ruin their business.", "Paul's evangelistic success", 'ruin their business')]
i: 70
out :  {
  "cause1": "the ointment",
  "effect1": "discomfort",
  "causality_phrase1": "caused by",
  "cause2": "the duration of treatment",
  "effect2": "non-compliance",
  "causality_phrase2": "and the duration of t

out :  {
  "cause": "the author's vast amount of material",
  "effect": "shallow generalizations",
  "causality_phrase": "to force him into"
}
empty
sent_cause_effect: [('The author clearly got a great deal of  pleasure  from  the work  and did not allow his vast amount of material to force him into shallow generalizations.', "the author's vast amount of material", 'shallow generalizations')]
i: 90
out :  {
  "cause1": "the relatively low systemic arterial blood pressure",
  "effect1": "Bounding pulses",
  "causality_phrase1": "are caused by",
  "cause2": "the continuous runoff of blood from the aorta into the pulmonary artery",
  "effect2": "",
  "causality_phrase2": "due to"}
sent_cause_effect: [(' Bounding pulses  are caused by  the relatively low systemic arterial blood pressure  due to  the continuous runoff  of blood from the aorta into the pulmonary artery.', 'the relatively low systemic arterial blood pressure', 'Bounding pulses'), (' Bounding pulses  are caused by  the relativ

out :  {
  "cause1": "the falling interest-rate structure caused by bond speculation",
  "effect1": "a victim",
  "causality_phrase1": "of",
  "cause2": "the falling interest-rate structure",
  "effect2": "the clandestine wealth-transfer plaguing the productive sector",
  "causality_phrase2": "as a result of"}
sent_cause_effect: [('Swissair was  a victim  of the clandestine wealth-transfer plaguing the productive sector as a result of  the falling interest-rate structure  caused by  bond speculation .', 'the falling interest-rate structure caused by bond speculation', 'a victim'), ('Swissair was  a victim  of the clandestine wealth-transfer plaguing the productive sector as a result of  the falling interest-rate structure  caused by  bond speculation .', 'the falling interest-rate structure', 'the clandestine wealth-transfer plaguing the productive sector')]
i: 113
out :  {
  "cause1": "the earthquake",
  "effect1": "the fire",
  "causality_phrase1": "after",
  "cause2": "the fire",
  

out :  {"cause": "the selling of crude oil in 2003", "effect": "Gross revenues reached US$11.508 billion or some 112.85 percent of the amount targeted in the 2003 state budget", "causality_phrase": "from"}
empty
sent_cause_effect: [(' Gross revenues  from  the selling  of crude oil in 2003 reached US$11.508 billion or some 112.85 percent of the amount targeted in the 2003 state budget.', 'the selling of crude oil in 2003', 'Gross revenues reached US$11.508 billion or some 112.85 percent of the amount targeted in the 2003 state budget')]
i: 139
out :  {
  "cause": "the explosion",
  "effect": "a gas leak on the pipeline",
  "causality_phrase": "caused"
}
empty
sent_cause_effect: [(' The explosion  caused  a gas leak  on the pipeline near the village of Pouce Coupe, south of Dawson Creek.', 'the explosion', 'a gas leak on the pipeline')]
i: 140
out :  {
  "cause1": "the bacterium Helicobacter pylori",
  "effect1": "inflammation in the stomach as well as ulceration of the stomach or duode

out :  {
  "cause": "the sunrise",
  "effect": "The light in the background",
  "causality_phrase": "is from"
}
empty
sent_cause_effect: [(' The light  in the background is from  the sunrise .', 'the sunrise', 'The light in the background')]
i: 164
out :  {
  "cause": "Trichophyton tonsurans",
  "effect": "ringworm of the scalp",
  "causality_phrase": "are caused by"
}
empty
sent_cause_effect: [('Over 90% of the cases of  ringworm  of the scalp are caused by  Trichophyton tonsurans , a fungus that infects the hairs and causes them to break.', 'Trichophyton tonsurans', 'ringworm of the scalp')]
i: 165
out :  {"cause": "the latest Christmas terrorist", "effect": "disruption", "causality_phrase": "caused by"}
empty
sent_cause_effect: [(' The disruption  caused by  the latest Christmas terrorist  has added too much to the weary trip home.', 'the latest Christmas terrorist', 'disruption')]
i: 166
out :  {
  "cause": "pressure",
  "effect": "Bed sores",
  "causality_phrase": "are caused by"


out :  {
  "cause1": "illnesses",
  "effect1": "a toxic overload",
  "causality_phrase1": "cause",
  "cause2": "a toxic overload",
  "effect2": "increases the stress on the kidneys",
  "causality_phrase2": "that also"}
sent_cause_effect: [('Most illnesses, including  colds  and  flu , cause  a toxic overload  that also increases the stress on the kidneys.', 'illnesses', 'a toxic overload'), ('Most illnesses, including  colds  and  flu , cause  a toxic overload  that also increases the stress on the kidneys.', 'a toxic overload', 'increases the stress on the kidneys')]
i: 187
out :  {
  "cause1": "intoxication from alcohol or illicit drug use",
  "effect1": "a social work assessment before leaving the hospital",
  "causality_phrase1": "had"}
sent_cause_effect: [('Every child and young person seen at Starship with  intoxication  from  alcohol or illicit drug use  had a social work assessment before leaving the hospital.', 'intoxication from alcohol or illicit drug use', 'a social work as

In [12]:
len(filtered_rows)

191

In [13]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

0

In [14]:
filtered_rows['sent_cause_effect'] = sent_cause_effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,embedings,sent_cause_effect,causality_phrase
0,0,0,Using solar electricity instead of conventiona...,"[0.030286945402622223, 0.00630235718563199, -0...",[(Using solar electricity instead of conventio...,"[helps to reduce, that causes]"
1,1,1,"Thus, evaluating capital punishment as a form ...","[-0.011410091072320938, 0.005278998054563999, ...","[(Thus, evaluating capital punishment as a for...",[that result in]
2,2,2,Sudden death from inhalation of petroleum ...,"[-0.0007025353843346238, 0.00263791810721159, ...",[( Sudden death from inhalation of petroleu...,[well recognised in misuses of volatile substa...
3,3,3,Stimulation of the posterior horn of the med...,"[-0.03430342674255371, 0.01450358610600233, 0....",[( Stimulation of the posterior horn of the m...,[produced]
4,4,4,Sip the tea slowly to reduce stomach pain fr...,"[0.005228457972407341, -0.011144616641104221, ...",[(Sip the tea slowly to reduce stomach pain ...,[from]
...,...,...,...,...,...,...
186,186,186,The disruption has been caused by work bei...,"[-0.019408993422985077, -0.003347919089719653,...",[( The disruption has been caused by work b...,[has been caused by]
187,187,187,"Most illnesses, including colds and flu , c...","[0.007080608978867531, -0.004280312918126583, ...","[(Most illnesses, including colds and flu ,...","[cause, that also]"
188,188,188,Every child and young person seen at Starship ...,"[0.019172143191099167, 0.013259614817798138, 0...",[(Every child and young person seen at Starshi...,[had]
189,189,189,The movie gives the inaccurate impression ...,"[-0.04086083546280861, -0.011357069946825504, ...",[( The movie gives the inaccurate impression...,[gives]


In [15]:
#SCITE

pm_DB = pd.read_csv('data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUTkeys.csv')
pm_DB['keys'] = pm_DB['keys'].apply(eval)



C_t = pm_DB['keys'].to_list()


C_p = sent_cause_effect_list

In [16]:
len(C_t)

296

In [17]:
C_p = [tup for sublist in C_p for tup in sublist]
#len(C_p[1])


In [18]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [19]:
len(C_p)

208

In [20]:
prediction= []
pred_label = []
for i in range(len(C_p)):
    #print(i)
    labeled = 0   
    if (C_p[i] != []):
        if len(C_p[i]) == 3:           
            
            sent = C_p[i][0]
            ####print('sent: ',sent )
            cause_pred = C_p[i][1]
            effect_pred = C_p[i][2]
            
            for i_t in range(len(C_t)):
                #get index of all tuples which match the sent and create sent_found = []
                # List to store indices
                sent_found = []

                # Check if sent is in the C_t at index 0
                for index, (sentence, _, _) in enumerate(C_t):
                    if sentence == sent:
                        sent_found.append(index)
            ####print('sent_found: ', sent_found)
            
            
            #iterate through sent_found and pass each cause_t and effect_t to get score: 
            for ind in sent_found:
                cause_t = C_t[ind][1]
                effect_t = C_t[ind][2]
                pred_c=0
                pred_e=0
                ##print('cause_t: ', cause_t)
                ##print('effect_t: ', effect_t)
                ##print('cause_pred: ', cause_pred)
                ##print('effect_pred: ', effect_pred)
                
                score = get_score(cause_t.lower(), cause_pred.lower())################ non -medcause
                    #score = fuzz.ratio(cause_pred, cause_t)################medcause
                if score >90:################90 non -medcause
                    pred_c = 1
                else:
                    pred_c=0
                    


                score = get_score(effect_t.lower(), effect_pred.lower())################non - medcause
                    #score = fuzz.ratio(effect_pred, effect_t)################medcause

                if score >90: ################90 non -medcause
                    pred_e = 1
                else:
                    pred_e=0



                if ((pred_e ==1) & (pred_c ==  1)):
                    prediction.append(1)
                    
                    labeled = 1
                    #pred_label.append(labeled)
                    #print('out:', 1)
    
    
    
    
    pred_label.append(labeled)
                    
    print(labeled, i)



                
    

 

0 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
0 10
1 11
no pred_phrase
no pred_phrase
0 12
1 13
1 14
prediction length lower headache p: a headache
0 15
prediction length lower intense discussion p: an intense discussion
prediction length lower accolade p: the accolade
0 16
prediction length lower about 200 members p: an intense discussion
no pred_phrase
0 17
0 18
1 19
1 20
prediction length lower the wind p: the steam
prediction length lower the wind p: a backpressure
0 21
prediction length lower pressure p: the pressure
0 22
1 23
prediction length lower discomfort p: enough discomfort
0 24
1 25
prediction length lower are intolerable p: sleep deprivation
0 26
prediction length lower the sun p: environmental pollutants
1 27
1 28
0 29
prediction length lower the drone strike p: deliberately planted false intelligence
prediction length lower a result of p: the drone strike
0 30
1 31
1 32
1 33
1 34
1 35
1 36
prediction length lower electricity p: the electricity
0 37
prediction length lower fo

In [21]:
prec=sum(prediction)/len(C_p)
rec = sum(prediction)/len(C_t)
F1 = 2 *(prec * rec)/(prec + rec)
print('F1: ', F1)
print('prec: ', prec)
print('rec: ', rec)


F1:  0.6626984126984127
prec:  0.8028846153846154
rec:  0.5641891891891891


In [22]:
len(pred_label)

208

In [23]:
len(C_p)

208

In [24]:
SCITE_df = pd.DataFrame({'pred_label': pred_label, 'C_p' : C_p})
SCITE_df

,pred_label,C_p
0,0,(Using solar electricity instead of convention...
1,1,(Using solar electricity instead of convention...
2,1,"(Thus, evaluating capital punishment as a form..."
3,1,( Sudden death from inhalation of petroleum...
4,1,( Stimulation of the posterior horn of the me...
...,...,...
203,0,"(Most illnesses, including colds and flu , ..."
204,0,"(Most illnesses, including colds and flu , ..."
205,0,(Every child and young person seen at Starship...
206,1,( The movie gives the inaccurate impression ...


In [25]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#fewshot_semeval_CE_detection
#SCITE_df.to_csv('data/out/RAG_causality/task2/gemma/RAG_SCITE_CE_extract.csv')
#filtered_rows.to_csv('data/out/RAG_causality/task2/GPT4/zero_sem_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [26]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_
